# Discrete Dynamic Programming

Pablo Winant

## Markov Chains

A worker’s employment dynamics is described by the stochastic matrix

$$P = \begin{bmatrix}
1-\alpha & \alpha \\
\beta & 1-\beta
\end{bmatrix}$$

with $\alpha\in(0,1)$ and $\beta\in (0,1)$. First line corresponds to
employment, second line to unemployment.

**Which is the stationary equilibrium? (choose any value for $\alpha$
and $\beta$)**

In [3]:
α = 0.2
β = 0.3
P =  [(1-α) α;  β (1-β)]

2×2 Matrix{Float64}:
 0.8  0.2
 0.3  0.7

In [16]:
# we want to find  μ P = μ 
# we can rewrite as P' μ' = μ'

# and solve (P' - I) x = 0

# identity operator:
using LinearAlgebra: I
M = (P'-I)

2×2 Matrix{Float64}:
 -0.2   0.3
  0.2  -0.3

In [8]:
# cannot the system directly because M is not inversble
# one row is colinear to the others
# M\zeros(2)#

In [9]:
# let' s replace one row of M with the condition |μ|=1

In [17]:
M[end,:] = ones(2) # set equal to vector
M[end,:].= 1.0      # set equal to a scalar (with vectorization)

2-element view(::Matrix{Float64}, 2, :) with eltype Float64:
 1.0
 1.0

In [19]:
M

2×2 Matrix{Float64}:
 -0.2  0.3
  1.0  1.0

In [28]:
[0.0 1.0] # one row vecto

1×2 Matrix{Float64}:
 0.0  1.0

In [29]:
[0.0, 1.0]

2-element Vector{Float64}:
 0.0
 1.0

In [21]:
μ = M \ [0.0, 1.0]

2-element Vector{Float64}:
 0.6000000000000001
 0.39999999999999997

In [26]:
# check μ is indeed the right solution
println(μ' * P - μ')
sum(μ)

[0.0 5.551115123125783e-17]

1.0

Other solution

In [33]:
MM = cat(
    P'-I,
    ones(1,2);
    dims=1
)

3×2 Matrix{Float64}:
 -0.2   0.3
  0.2  -0.3
  1.0   1.0

In [34]:
MM \ [0.0, 0.0, 1.0]

2-element Vector{Float64}:
 0.6
 0.3999999999999999

**In the long run, what will the the fraction $p$ of time spent
unemployed? (Denote by $X_m$ the fraction of dates were one is
unemployed)**

**Illustrate this convergence by generating a simulated series of length
10000 starting at $X_0=1$. Plot $X_m-p$ against $m$. (Take
$\alpha=\beta=0.1$).**

In [35]:
using Plots

## Job-Search Model

We want to solve the following model, adapted from McCall.

-   When unemployed in date, a job-seeker
    -   consumes unemployment benefit $c_t = \underline{c}$
    -   receives in every date $t$ a job offer $w_t$
        -   $w_t$ is i.i.d.,
        -   takes values $w_1, w_2, w_3$ with probabilities
            $p_1, p_2, p_3$
    -   if job-seeker accepts, becomes employed at rate $w_t$ in the
        next period
    -   else he stays unemployed
-   When employed at rate $w$
    -   worker consumes salary $c_t = w$
    -   with small probability $\lambda>0$ looses his job:
        -   starts next period unemployed
    -   otherwise stays employed at same rate
-   Objective: $\max E_0 \left\{ \sum \beta^t \log(w_t) \right\}$

**What are the states, the controls, the reward of this problem ? Write
down the Bellman equation.**

cf course

**Define a parameter structure for the model.**

In [1]:
model = (;
    λ=0.01,
    cbar=0.8,
    β=0.9,
    wvec=[0.9, 1.0, 1.1],
    pvec=[1/3,1/3,1/3]
)

(λ = 0.01, cbar = 0.8, β = 0.9, wvec = [0.9, 1.0, 1.1], pvec = [0.3333333333333333, 0.3333333333333333, 0.3333333333333333])

**Define a function
`value_update(V_U::Vector{Float64}, V_E::Vector{Float64}, x::Vector{Bool}, p::Parameters)::Tuple{Vector, Vector}`,
which takes in value functions tomorrow and a policy vector and return
updated values for today.**

In [6]:
V_U_0 = zeros(3)
V_E_0 = zeros(3)
x_0 = [false, false, false]

function value_update(V_U, V_E, x, model)

    (;cbar, λ, β) = model
    n_V_E = zeros(3)
    n_V_U = zeros(3)

    c_V_U = sum( p*V_U[j] for (j,p) in  enumerate(model.pvec ))

    for i=1:3
        n_V_E[i] = log(model.wvec[i]) + (1-λ)*β*V_E[i] + λ*β*c_V_U
    end

    for i=1:3
        if x[i]
            n_V_U[i] = log(model.cbar) + β*V_E[i]
        else
            n_V_U[i] = log(model.cbar) + β*c_V_U
        end
    end

    return n_V_U, n_V_E
end

value_update(V_U_0, V_E_0, x_0, model)

([-0.2231435513142097, -0.2231435513142097, -0.2231435513142097], [-0.10536051565782628, 0.0, 0.09531017980432493])

**Define a function
`policy_eval(x::Vector{Bool}, p::Parameter)::Tuple{Vector, Vector}`
which takes in a policy vector and returns the value(s) of following
this policies forever. You can add relevant arguments to the function.**

In [10]:
function policy_eval(x, model; T=500, tol_η=1e-6)

    V_U_0 = zeros(3)
    V_E_0 = zeros(3)

    for t=1:T
        V_U, V_E = value_update(V_U_0, V_E_0, x, model)
        η = maximum(abs.( [  V_U-V_U_0   ;  V_E-V_E_0   ]))
        if η<tol_η
            return V_U, V_E
        end

        V_U_0, V_E_0 = V_U, V_E
    end

    
end

policy_eval (generic function with 1 method)

In [13]:
policy_eval([false, false, false], model)

([-2.2314266170257446, -2.2314266170257446, -2.2314266170257446], [-1.1508496241589632, -0.18424056588532145, 0.6901636885865156])

In [17]:
policy_eval([false, true, true], model)

([-0.16894144584658277, -0.21867165307166767, 0.5682888378093481], [-0.9616363330197235, 0.0049692003147929036, 0.8793702660913234])

**Define a function
`bellman_step(V_E::Vector, V_U::Vector, p::Parameters)::Tuple{Vector, Vector, Vector}`
which returns updated values, together with improved policy rules.**

In [18]:

function bellman_step(V_U, V_E, model)

    (;cbar, λ, β) = model
    n_V_E = zeros(3)
    n_V_U = zeros(3)
    x = zeros(Bool, 3)

    c_V_U = sum( p*V_U[j] for (j,p) in  enumerate(model.pvec ))

    for i=1:3
        n_V_E[i] = log(model.wvec[i]) + (1-λ)*β*V_E[i] + λ*β*c_V_U
    end

    # when unemployed
    for i=1:3
        v_accept = log(model.cbar) + β*V_E[i]
        v_reject = log(model.cbar) + β*c_V_U
        if v_accept>v_reject
            n_V_U[i] = v_accept
            x[i] = true
        else
            n_V_U[i] = v_reject
            x[i] = false
        end
    end

    return n_V_U, n_V_E, x
end

bellman_step (generic function with 1 method)

In [20]:
bellman_step(V_U_0, V_E_0, model)

([-0.2231435513142097, -0.2231435513142097, -0.2231435513142097], [-0.10536051565782628, 0.0, 0.09531017980432493], Bool[0, 0, 0])

**Implement Value Function**

In [25]:
function vfi(model; T=500, tol_η=1e-6)

    V_U_0 = zeros(3)
    V_E_0 = zeros(3)

    for t=1:T
        V_U, V_E, x = bellman_step(V_U_0, V_E_0, model)
        η = maximum(abs.( [  V_U-V_U_0   ;  V_E-V_E_0   ]))
        println("$t : $η")
        if η<tol_η
            return V_U, V_E, x
        end

        V_U_0, V_E_0 = V_U, V_E
    end

    
end

vfi (generic function with 2 methods)

In [26]:
V_U, V_E, x = vfi(model)

1 : 0.2231435513142097
2 : 0.09588451141295112
3 : 0.08629606027165598
4 : 0.07618551515584138
5 : 0.06776620825824747
6 : 0.06017209711438726
7 : 0.05339623450323355
8 : 0.04737184303259734
9 : 0.042021967211819056
10 : 0.037273116454005106
11 : 0.033058519006658904
12 : 0.029318432293365393
13 : 0.025999667465517406
14 : 0.023054948225977645
15 : 0.020442277375651963
16 : 0.018124355344447007
17 : 0.01606805806256495
18 : 0.014243971400986988
19 : 0.01262597700763557
20 : 0.011258315445731748
21 : 0.010085339325077824
22 : 0.009048942948106098
23 : 0.008127581948617513
24 : 0.007305091931290697
25 : 0.006568831231085204
26 : 0.005908548967294203
27 : 0.005315685178421625
28 : 0.004782929405322878
29 : 0.004303934796933695
30 : 0.0038731266315371604
31 : 0.003485568889132873
32 : 0.0031368671583823504
33 : 0.002823094841018392
34 : 0.0025407347664148094
35 : 0.0022866313907867752
36 : 0.0020579505814068977
37 : 0.0018521450801185502
38 : 0.0016669244002061046
39 : 0.001500228312592444

([-0.12918217853332556, -0.12918217853332556, 0.5715717530233522], [-0.9579912751556052, 0.008615634245837868, 0.88301794482632], Bool[0, 0, 1])

In [27]:
x

3-element Vector{Bool}:
 0
 0
 1

**Implement Policy Iteration and compare rates of convergence.**

In [31]:
function policy_iteration(model; T=500, tol_η=1e-6)

    V_U_0 = zeros(3)
    V_E_0 = zeros(3)

    for t=1:T
        V_U, V_E, x = bellman_step(V_U_0, V_E_0, model)
        η = maximum(abs.( [  V_U-V_U_0   ;  V_E-V_E_0   ]))
        println("$t : $η : $x")
        if η<tol_η
            return V_U, V_E, x
        end

        V_U_0, V_E_0 = policy_eval(x, model)
    end

    
end

policy_iteration (generic function with 1 method)

In [32]:
V_U, V_E, x = policy_iteration(model)

1 : 0.2231435513142097 : Bool[0, 0, 0]
2 : 2.6294303854393988 : Bool[1, 1, 1]
3 : 0.6462314803394785 : Bool[0, 1, 1]
4 : 0.04973082342478724 : Bool[0, 0, 1]
5 : 8.853353918869544e-7 : Bool[0, 0, 1]

([-0.1291816864905906, -0.1291816864905906, 0.5715722450660872], [-0.9579914735969917, 0.00861579833752028, 0.883018436869055], Bool[0, 0, 1])

**Discuss the Effects of the Parameters**

In [36]:
policy_iteration(model)[3]

1 : 0.2231435513142097 : Bool[0, 0, 0]
2 : 2.6294303854393988 : Bool[1, 1, 1]
3 : 0.6462314803394785 : Bool[0, 1, 1]
4 : 0.04973082342478724 : Bool[0, 0, 1]
5 : 8.853353918869544e-7 : Bool[0, 0, 1]

3-element Vector{Bool}:
 0
 0
 1

In [40]:
policy_iteration(merge(model, (;cbar=0.3)))[3]

1 : 1.2039728043259361 : Bool[0, 0, 0]
2 : 10.728021688918453 : Bool[1, 1, 1]
3 : 8.015472237055121e-7 : Bool[1, 1, 1]

3-element Vector{Bool}:
 1
 1
 1

In [ ]:
merge(model, (;cbar=0.7)